### FE-620 FINAL PROJECT: BINOMIAL PRICING MODEL FOR SPY OPTIONS

#### IMPORTS

In [871]:
from datetime import datetime
import datetime as dt
import yfinance as yf
import pandas as pd
import numpy as np
import copy
import os

#### PARAMETERIZATION FUNCTIONS

In [872]:
def _get_binom_rn_params(n : float, t : float, r : float, hvol : float, prec : int):
    # calc tree params
    dt = t/n
    u = np.exp(hvol*np.sqrt(dt))
    d = 1/u
    p = (np.exp(r*dt)-d)/(u-d)
    q = 1 - p
    # print divider
    print("-"*80)
    # print params
    print((f"dt: {round(dt, 2)}  | u: {round(u, 2)}  |  d: {round(d, 2)}  |  p: {round(p, 2)}  |  q: {round(q, 2)}  |  hvol: {round(hvol, 2)}").center(80)) # 56 is len of divider
    # print divider
    print("-"*80)
    return t, dt, u, d, p, q

In [873]:
def _get_div_idx(div : dict, steps : int, maturity : float):
    if div is None:
        return None
    # print dividends
    print(f"dividends: {str(div)}".center(80))
    # get "todays" date - time 0
    start = list(div.keys())[0]
    # copy dict so dont change during iteration
    idxdiv = copy.deepcopy(div)
    # replace start date with index 0 for start date
    idxdiv[0] = idxdiv.pop(start)
    # calc time length of each step
    dt = maturity/steps
    # get index of each div date 
    for date in list(div.keys()): # use list() to avoid dict size change during iteration
        # skip start key/val pair
        if date != start:
            # calculate index aka days since start
            idx = round((np.busday_count(np.datetime64(start), np.datetime64(date))/252)/dt)
            # switch date with index
            idxdiv[idx] = idxdiv.pop(date)
    return idxdiv

# explanation of dividend index calculation for varied time steps:
# diff days / 252 = diff in years since start
# dt = (t in years) / steps = time of step in years
# diff in years / time of step in years = steps since start -> rounded = index of step


#### PRICING FUNCTIONS

In [874]:
def _calc_maturity_value(stock : list, tree : list, s0 : float, k : float, n : int, t : float, u : float, d : float, prec : int, type : str, div : dict):
    # call div idx func if divs - will return div to model for subsequent backprop call
    idxdiv = _get_div_idx(div, n, t)
    # if idxdiv is not None: print(f"dividends @ indices: {idxdiv}".center(80))
    # print divider
    print("-"*80)
    # print header for output
    print("ADJUSTMENT & EXERCISE REPORT".center(80))
    # print divider
    print("-"*80)
    # check if dividend paid at maturity by checking if # of total steps is in div dict
    if idxdiv is not None and n in list(idxdiv.keys()):
        # get div amount
        div_adj = idxdiv[n]
        # print div adj
        print(f"time {n}: dividend adjustment = {div_adj}")
    else:
        div_adj = 0
    # check option type
    if type.lower() == "call":
        # iterate over rows (price/nodes) @ maturity
        for i in range(n, -1, -1):
            # calc stock price @ maturity considering potential dividend
            stock[n][i] = round((s0*(u**i)*(d**(n-i)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves - div adj
            # print stock price before & after div adj
            if div_adj != 0: print(f"time {n}: before div adj @ ${stock[n][i]+div_adj} & after div adj @ ${stock[n][i]}")
            # calc option price @ maturity considering potential dividend
            tree[n][i] = round(max(stock[n][i]-k, 0.0), prec) # max(s - d - k, 0)
    elif type.lower() == "put":
        # iterate over rows (price/nodes) @ maturity
        for i in range(n, -1, -1):
            # calc stock price @ maturity considering potential dividend
            stock[n][i] = round((s0*(u**i)*(d**(n-i)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves - div adj
            # print stock price before & after div adj
            if div_adj != 0: print(f"time {n}: before div adj @ ${stock[n][i]+div_adj} & after div adj @ ${stock[n][i]}")
            # calc option price @ maturity considering potential dividend
            tree[n][i] = round(max(k-stock[n][i], 0.0), prec) # max(k - s, 0)
    else:
        raise ValueError("invalid option type - please enter 'call' or 'put'")
    return stock, tree, idxdiv
    

In [875]:
def _back_prop(stock : list, tree : list, idxdiv : dict, s0 : float, k : float, n : int, dt : float, r : float, u : float, d : float, p : float, q : float, prec : int, type : str):
    # iterate over cols (time)
    for i in range(n-1, -1, -1):
        # dividend idxs have already been identified in the maturity calculation so just have to check for them
        # check if div paid at this time step
        if idxdiv is not None and i in list(idxdiv.keys()) and i != 0:
            # get div amount
            div_adj = idxdiv[i]
            # print div adj
            print(f"time {i}: dividend adjustment of - ${div_adj}".center(80))
        else:
            div_adj = 0
        # iterate over rows (price/nodes)
        for j in range(i+1, -1, -1):
            # fill in stock price at node
            stock[i][j-1] = round((s0*(u**j)*(d**(i-j)))-div_adj, prec) # s = s0 x up % ^ # up moves x down % ^ # down moves
            # find option price at node by discounting expected value of option price at next (later) time step
            tree[i][j-1] = round(np.exp(-r*dt)*(p*tree[i+1][j]+q*tree[i+1][j-1]), prec)
            if type.lower() == "call":
                # check exercise for call (option price vs intrinsic value) 
                if max(tree[i][j-1], (stock[i][j-1] - k)) == (stock[i][j-1] - k):
                    print(f"time {i}: call exercised @ price ${stock[i][j-1]}".center(80))
                tree[i][j-1] = round(max(tree[i][j-1], (stock[i][j-1] - k)), 2) # max(s - k, 0)
            else:
                # check exercise for put (option price vs intrinsic value)
                if max(tree[i][j-1], (k - stock[i][j-1])) == (stock[i][j-1] - k):
                    print(f"time {i}: put exercised @ price ${stock[i][j-1]}".center(80))
                tree[i][j-1] = round(max(tree[i][j-1], k - stock[i][j-1]), 2) # max(k - s, 0)
    return stock, tree

#### PRINTING FUNCTIONS

In [876]:
def _print_model(type : str):
    # print divider
    print("-"*80)
    # print model
    print((f"AMERICAN {type.upper()} OPTION - RISK-NEUTRAL BINOMIAL PRICING MODEL").center(80)) # 56 is len of divider

In [877]:
def _print_stock(stock : list):
    # print divider
    print("-"*80)
    # print header
    print("STOCK PRICE TREE".center(80))
    # print divider
    print("-"*80)
    # print price
    for time, prices in enumerate(stock, 0):
        centered = str(prices).center(65) # 65 is len of divider after time #:
        print(f"time {time}: {centered}")

In [878]:
def _print_tree(tree : list, type : str):
    # print divider
    print("-"*80)
    # print header
    print(f"{type.upper()} OPTION PRICE TREE".center(80))
    # print divider
    print("-"*80)
    # print tree
    for time, branch in enumerate(tree, 0):
        # print centered row
        centered = str(branch).center(65) # 65 is len of divider after time #:
        print(f"time {time}: {centered}")
    # print divider
    print("-"*80)

DOWNLOAD DATA

In [879]:
def _get_hdata(ticker : str, hist : str, end : str):
    # check if data is already downloaded
    path = '../csvs/spy_historical.csv'
    if os.path.isfile(path):
        df = pd.read_csv(path)
    # otherwise download data
    else:
        # download data
        df = yf.download('SPY', start=hist, end=end)
        # save data
        df.to_csv(path, header=True)
    # set date to datetime
    df['Date'] = pd.to_datetime(df['Date'])
    # get adj close & convert index
    df = df[['Date', 'Adj Close']].rename(columns={'Adj Close':'spy', 'Date':'date'}).set_index('date')
    return df

#### HISTORICAL VOLATILITY FUNCTION

In [880]:
def _get_hvol(df, hist : str, start : str, prec : int):
    # convert dates to datetime
    hist, start = pd.to_datetime(hist), pd.to_datetime(start)
    # calc historical volatility for window & annualize
    return ((df['spy'].pct_change().loc[hist:start]).dropna().std()*np.sqrt(252)).round(prec)

#### BINOMIAL PRICING MODEL FOR AMERICAN OPTION USING RISK NEUTRAL PROBABILITY

In [881]:
def binom_rn_pricer(s0 : float, k : float, n : int, t : float, r : float, hvol : float, prec : int, type : str, div : dict):
    """
    s0: initial stock price
    k: strike price
    n: number of steps
    t: time to maturity per annum
    r: risk-free rate
    hvol: historical volatility - window size = maturity of rfr
    maturity: maturity of rate used (e.g, 1-month = 1, 3-month = 3, 6-month = 6, 1-year = 12)
    type: option type (call or put)
    div: dividend dictionary {start date : pricing date, div1 date : div1 #, div2 date : div2 #, ...}
    """
    # print model
    _print_model(type)
    # get adjusted & calculated params
    t, dt, u, d, p, q = _get_binom_rn_params(n, t, r, hvol, prec)
    # initialize option tree
    tree = [[0]*(i+1) for i in range(n+1)]
    # initialize stock tree
    stock = [[0]*(i+1) for i in range(n+1)]
    # calc option prices at maturity
    stock, tree, idxdiv = _calc_maturity_value(stock, tree, s0, k, n, t, u, d, prec, type, div)
    # backpropagate
    stock, tree = _back_prop(stock, tree, idxdiv, s0, k, n, dt, r, u, d, p, q, prec, type)
    # print option tree
    _print_tree(tree, type)
    # print stock price tree
    _print_stock(stock)
    # return price and df
    return tree[0][0], stock[0][0], tree, stock

#### PRICING TEST

THEORETICAL DUMMY DATA

In [882]:
s0 = 100 # market price as of 6/1/23
k = 101 # call strike
n = 3 # if daily -> biz days to expiration (6/1/23 - 11/17/23) = 122
t = 6/12 # tmt per annum
r = .05 # 5% per annum
hvol = .25 # 10% per annum
prec = 2 # precision
type = "call" # option type
div = None # dividend yield

In [883]:
optprice, stockprice, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=t, r=r, hvol=hvol, prec=prec, type=type, div=div)

--------------------------------------------------------------------------------
           AMERICAN CALL OPTION - RISK-NEUTRAL BINOMIAL PRICING MODEL           
--------------------------------------------------------------------------------
     dt: 0.17  | u: 1.11  |  d: 0.9  |  p: 0.52  |  q: 0.48  |  hvol: 0.25      
--------------------------------------------------------------------------------
--------------------------------------------------------------------------------
                          ADJUSTMENT & EXERCISE REPORT                          
--------------------------------------------------------------------------------
                     time 2: call exercised @ price $150.42                     
                     time 2: call exercised @ price $122.65                     
                     time 1: call exercised @ price $135.82                     
                     time 0: call exercised @ price $122.65                     
----------------------------

MARKET-LIKE DUMMY DATA

In [884]:
# date params
hist = '2022-06-01' # 1 year of historical data
start = '2023-06-01' # observation date (i.e, t = 0)
end = '2023-11-17' # expiration date (i.e, t = n)
# contract params
type = "call" # option type
tdays = np.busday_count(np.datetime64(start), np.datetime64(end))
t = tdays / 252 # tmt per annum
prec = 2 # precision
s0 = 421.82 # market price as of 6/1/23
k = 455 # strike
n = 6 # number of steps
# market params
r = .0517 # feed in appropriate rate
histdf = _get_hdata('SPY', hist, end) # get historical data
hvol = _get_hvol(histdf, hist, start, prec) # get historical volatility
div = {start : 0, "2023-06-16" : 1.64, "2023-09-15" : 1.58} # use ex-dividend dates

In [885]:
optprice, stockprice, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=t, r=r, hvol=hvol, prec=prec, type=type, div=div)

--------------------------------------------------------------------------------
           AMERICAN CALL OPTION - RISK-NEUTRAL BINOMIAL PRICING MODEL           
--------------------------------------------------------------------------------
     dt: 0.08  | u: 1.06  |  d: 0.94  |  p: 0.52  |  q: 0.48  |  hvol: 0.21     
--------------------------------------------------------------------------------
      dividends: {'2023-06-01': 0, '2023-06-16': 1.64, '2023-09-15': 1.58}      
--------------------------------------------------------------------------------
                          ADJUSTMENT & EXERCISE REPORT                          
--------------------------------------------------------------------------------
                     time 5: call exercised @ price $639.33                     
                     time 5: call exercised @ price $567.71                     
                     time 5: call exercised @ price $504.11                     
                     time 4:

#### READ & PULL DATA

In [886]:
call = pd.read_csv('../csvs/spycall.csv')
put = pd.read_csv('../csvs/spyput.csv')

In [887]:
call['date'] = pd.to_datetime(call['date'], format='%m/%d/%y')
put['date'] = pd.to_datetime(put['date'], format='%m/%d/%y')
call.set_index('date', inplace=True)
put.set_index('date', inplace=True)

### SET PARAMETERS SENSITIVITY ANALYSIS

PARAMS SPECIFIED FOR RUNS WHERE ALL OTHER HELD CONSTANT

In [889]:
# time params
hist = '2022-06-01' # 1 year of historical data
t0 = '2023-06-01' # observation date (i.e, t = 0)
end = '2023-11-17' # expiration date (i.e, t = n)
# market params
rate = call['sofr12m'].loc[datetime(2023, 6, 1, 0, 0)]  # get t = 0 rate
histdf = _get_hdata('SPY', hist, end) # get historical data
hvol = _get_hvol(histdf, hist, t0, prec) # get 1yr historical volatility
div = {start : 0, "2023-06-16" : 1.64, "2023-09-15" : 1.58} # use ex-dividend dates
# contract params
type = "call" # option type
tdays = np.busday_count(np.datetime64(t0), np.datetime64(end))
t = tdays / 252 # tmt per annum
s0 = 421.82 # market price as of 6/1/23
k = 455 # strike
n = 5                                           # TODO: decide which n to use
prec = 2 # precision

optprice, stockprice, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=t, r=rate, hvol=hvol, prec=prec, type=type, div=div)

--------------------------------------------------------------------------------
           AMERICAN CALL OPTION - RISK-NEUTRAL BINOMIAL PRICING MODEL           
--------------------------------------------------------------------------------
     dt: 0.1  | u: 1.07  |  d: 0.94  |  p: 5.29  |  q: -4.29  |  hvol: 0.21     
--------------------------------------------------------------------------------
      dividends: {'2023-06-01': 0, '2023-06-16': 1.64, '2023-09-15': 1.58}      
--------------------------------------------------------------------------------
                          ADJUSTMENT & EXERCISE REPORT                          
--------------------------------------------------------------------------------
                     time 4: call exercised @ price $480.45                     
                     time 4: call exercised @ price $325.14                     
                     time 3: dividend adjustment of - $1.58                     
                     time 3:

ITERATE OVER TIME - will use this to call on later functions over time to show relationship

In [890]:
# at t=0, 3m tmt, 1m tmt, 1w tmt
# 119 - #*5 for date before # weeks
startdates = list(range(1:round(len(call.index)/5))) # weeks before
# TODO: get start dates for 3m, 1m, 1w and then call binom_rn_pricer for each


# get last 6 months, last 3 months, and last 1 month
start3 = np.busday_offset(start, 3*5, roll='forward') # forward = next biz days for non-biz day calc
hvol3 = np.busday_offset(start, 2*5, roll='forward')
hvol1 = np.busday_offset(start, 1*5, roll='forward')
# swith to datetime for string conversion
hvol6 = hvol6.astype(datetime)
hvol3 = hvol3.astype(datetime)
hvol1 = hvol1.astype(datetime)

STEPS

In [ ]:
# TODO try different step values with all else equal to see how it affects price

RATES

In [891]:
# call.ravg12m.plot()
def get_rates(df, ratecol : str):
    rates = []
    # get stats
    result = df[ratecol].describe()
    # get testing rates
    for stat, value in result.items():
        # add all stats except count, std, & max
        if stat == 'count' or stat == 'std' or stat == 'max':
            continue
        rates.append(round(value, prec))
    # return rates
    return rates

get_rates(call, 'sofr12m')

[5.35, 5.03, 5.3, 5.37, 5.41]

In [ ]:
def _pop_varied_rates_prices(df, ratecols : [str], tmt : float, type : str):
    for ratecol in ratecols:
        # get rates
        rate = get_rates(df, ratecol)
            optprice, stockprice, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=tmt, r=rate, hvol=hvold, prec=prec, type=type, div=div)
        df[date][f'{ratecol}price'] = optprice

ratecols = ['sofr12m', 'sofr6m', 'sofr3m', 'sofr1m', 'sofr1w']
_pop_varied_hvol_prices(call, ratecols, tmt, type)

HISTORICAL VOL TIME WINDOWS

In [892]:
def _get_hvol_dates(hist : str, start : str):
    # get last 6 months, last 3 months, and last 1 month
    hvol6 = np.busday_offset(hist, 6*21, roll='forward') # forward = next biz days for non-biz day calc
    hvol3 = np.busday_offset(hist, 9*21, roll='forward')
    hvol1 = np.busday_offset(hist, 11*21, roll='forward')
    # swith to datetime for string conversion
    hvol6 = hvol6.astype(datetime)
    hvol3 = hvol3.astype(datetime)
    hvol1 = hvol1.astype(datetime)
    # add string versions to dict
    hvols = {'hvol6' : hvol6.strftime('%Y-%m-%d'), 'hvol3' : hvol3.strftime('%Y-%m-%d'), 'hvol1' : hvol1.strftime('%Y-%m-%d')}
    return hvols
print(_get_hvol_dates(hist, start))

{'hvol6': '2022-11-24', 'hvol3': '2023-02-21', 'hvol1': '2023-04-20'}


POPULATE THE DF W/ PRICES FROM VARIED PARAMS

In [893]:

def _pop_varied_hvol_prices(df, hist : str, start : str, prec : int, tmt : float, type : str):
    hvols = _get_hvol_dates(hist, start)
    for volw, date in hvols.items:
        # get hvol for each date window
        hvold = _get_hvol(histdf, date, start, prec)
        # price
        optprice, stockprice, tree, stock = binom_rn_pricer(s0=s0, k=k, n=n, t=tmt, r=rate, hvol=hvold, prec=prec, type=type, div=div)
        df[date][f'{volw}price'] = optprice

# _pop_varied_hvol_prices(histdf, hist, start, prec, t, type)

IV WINDOW

PRICE TREE PARAMETERS USING HISTORICAL CALIBRATION

In [894]:
def calculate_up_down_params(df):
    # get returns
    returns = df['spy'].pct_change().dropna()
    # get up & down moves
    up_moves = returns[returns > 0]
    down_moves = returns[returns < 0]
    # calc params
    avg_up = up_moves.mean()
    avg_down = down_moves.mean()

    p_up = len(up_moves) / len(returns)
    p_down = len(down_moves) / len(returns)

    return avg_up, avg_down, p_up, p_down


ITERATE OVER TMT

- check pricing model (iterate through each day from t=1 to t=T pricing using that day's tmt & s0 (feed in daily rate but run on different sofrs] to see how tmt & rate affects accuracy)
- plot both model's stock prices vs market, model's option prices vs market, 
- run sensitivty analysis and save results for graphs

^^^ basically just need to turn the 'INITIALIZE MODEL PARAMETERS' into variable and feed in stock data 